In [ ]:
%load_ext autoreload

In [ ]:
import json
import pickle

import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
from scipy import spatial
import scipy.sparse as sp
from itertools import islice
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec

import sys
sys.path.append('../../')

import src.test.util as test_util

from src.packages.TPPMI.ppmi_model import PPMIModel
from src.packages.TPPMI.tppmi_model import TPPMIModel

# Imports

In [ ]:
path_to_test_data = Path("../../data") / "test"
path_to_tppmi_model = Path("../../data") / "ppmi-matrices" / "nyt-data"
path_to_twec_model = Path("../../model") / "nyt-data" / "cade" / "model"
path_to_static_model = Path("../../model") / "nyt-data" / "static"

# Functions

(to be outsourced later)

## Similarity Calculations

Create dictionary of the test-set for each particular model

In [ ]:
def create_test_case_dict_cade(test_cases : np.ndarray, models: dict) -> dict:
    test_case_dict = dict()
    counter = 0
    for test_case in test_cases:
        word, year = test_case.split("-")
        ground_model = models[f"model_{year}"]
        if word in ground_model.wv.vocab:
            test_case_dict[test_case] = ground_model.wv.get_vector(word)
        else:
            counter = counter + 1
    print(f"{counter} Testcases are not in the vocab of the model(s)")
    return test_case_dict

In [ ]:
def create_test_case_dict_static(model, test_cases : np.ndarray) -> dict:
    test_case_dict = dict()
    counter = 0
    for test_case in test_cases:
        word, year = test_case.split("-")
        if word in model.wv.vocab:
            test_case_dict[test_case] = model.wv.get_vector(word)
        else:
            counter = counter + 1
    print(f"{counter} Testcases are not in the vocab of the model")
    return test_case_dict

Calculate the top-10 most similar words to each word-vector/word-embedding from the test-set in each year

In [ ]:
def get_similarities_of_models(model_dict: dict, test_word_dict: dict) -> dict:
    similarities = dict()
    for test_word in tqdm(test_word_dict.items()):
        similarities[test_word[0]] = dict()
        for model in model_dict.items():
            similarities[test_word[0]][model[0].split("_")[1]] = model[1].wv.similar_by_vector(test_word[1])
    return similarities

In [ ]:
def get_similarities_of_models_static(model, test_word_dict: dict) -> dict:
    similarities = dict()
    for test_word in tqdm(test_word_dict.items()):
        similarities[test_word[0]] = model.wv.similar_by_vector(test_word[1])
    return similarities

## Evaluation Metrics

In [ ]:
def calculate_reciprocal_rank(test_list: list, test_word: str) -> float:
    """
    Calculate the reciprocal rank for a given test word in a list of strings.

    Parameters:
    test_list (list of str): The list of strings to search through.
    test_word (str): The correct answer to find in the test_list.
    Returns:
    float: The reciprocal rank of the test_word in test_list, or 0 if not found.
    """
    try:
        rank = test_list.index(test_word) + 1  # Adding 1 because index is 0-based and rank is 1-based
        return 1.0 / rank
    except ValueError:
        return 0.0  # test_word not found in test_list

In [ ]:
def calculate_precision_at_k(test_list: list, test_word: str, k: int) -> int:
    """
    Calculate the precision at K for a given test word in a list of strings.

    Parameters:
    test_list (list of str): The list of strings to search through, assumed to be ordered by relevance.
    test_word (str): The correct answer to find in the test_list.
    k (int): The number of top items to consider for calculating precision.

    Returns:
    int: The precision at K for the test_word in test_list.
         If the target word is among these K words, then the Precision@K for test i
         (denoted P@K[i]) is 1; else, it is 0
    """
    if k <= 0:
        raise ValueError("k must be a positive integer")

    # Take the top K elements from the list
    top_k = test_list[:k]

    # Check if the test_word is within the top K elements
    if test_word in top_k:
        return 1
    else:
        return 0

In [ ]:
def calculate_mean_rank(test_key: str, testcase: dict, test_data: pd.DataFrame, metric = "MRR", k = 10)-> float:

    test_data_for_key = test_data[test_data["truth"] == test_key]
    ranks = []

    for key, value in testcase.items():
        test_data_for_year = test_data_for_key[test_data_for_key["equivalent"].str.endswith(key)]
        word_list = [item[0] for item in value]

        if len(test_data_for_year) == 0:
            continue  # Skip if no data for year, as there's nothing to rank
        target_word = test_data_for_year["equivalent"].iloc[0].split("-")[0]
        if metric == "MRR":
            rank = calculate_reciprocal_rank(word_list, target_word)
        else:
            rank = calculate_precision_at_k(word_list, target_word, k)

        ranks.append(rank)

    if ranks:  # Ensure division by 0 does not occur
        mean_rank = sum(ranks) / len(ranks)
    else:
        mean_rank = 0

    return mean_rank

In [ ]:
def calculate_rank_metric(similarities: dict, test_data: pd.DataFrame, metric = "MRR", k = 10) -> float:
    ranks = []
    for key, value in similarities.items():
        rank = calculate_mean_rank(key, value, test_data, metric, k)

        ranks.append(rank)

    if ranks:  # Ensure division by 0 does not occur
        mean_rank = sum(ranks) / len(ranks)
    else:
        mean_rank = 0

    return mean_rank

In [ ]:
def calculate_rank_metric_static(similarities: dict, test_data: pd.DataFrame, metric = "MRR", k = 10) -> float:

    ranks = []
    counter = 0

    for key, value in similarities.items():
        test_data_for_key = test_data[test_data["truth"] == key]
        word_list = [item[0] for item in value]
        for _, test_case in test_data_for_key.iterrows():
            target_word = test_case["equivalent"].split("-")[0]
            if metric == "MRR":
                rank = calculate_reciprocal_rank(word_list, target_word)
            else:
                rank = calculate_precision_at_k(word_list, target_word, k)
            ranks.append(rank)
            counter = counter + 1


    if ranks:  # Ensure division by 0 does not occur
        mean_rank = sum(ranks) / len(ranks)
    else:
        mean_rank = 0

    return mean_rank

# Setup

## Testsets

### Testset 1

Based on publicly recorded knowledge that for each year lists different names for a particular role, such as U.S. president, U.K. prime minister, NFL superbowl champion team, and so on.

In [ ]:
test_data_1_all = pd.read_csv(path_to_test_data / "testset_1.csv")

In [ ]:
test_data_1_all.columns = ['truth', 'equivalent']

In [ ]:
test_data_1_all = test_data_1_all.sort_values(by='truth', ascending=True)

In [ ]:
test_cases_1_all = test_data_1_all['truth'].unique()

In [ ]:
print("Testset 1")
print(f"Testcases (all): {len(test_data_1_all)}")
print(f"Testcases (unique): {len(test_cases_1_all)}")

Now we want to split the testset into static & dynamic testcases as was done by Di Carlo et al. in their paper "Training Temporal Word Embeddings with a Compass"

In [ ]:
# Splitting the columns into words and years
split_truth = test_data_1_all['truth'].str.split('-', expand=True)
split_equivalent = test_data_1_all['equivalent'].str.split('-', expand=True)

# Creating masks for "static" and "dynamic" conditions
static_mask = split_truth[0] == split_equivalent[0]
dynamic_mask = split_truth[0] != split_equivalent[0]

# Applying the masks to create the separate DataFrames
test_data_1_static = test_data_1_all[static_mask]
test_data_1_dynamic = test_data_1_all[dynamic_mask]

In [ ]:
test_cases_1_static = test_data_1_static['truth'].unique()
test_cases_1_dynamic = test_data_1_dynamic['truth'].unique()

In [ ]:
print("Static")
print(f"Testcases (all): {len(test_data_1_static)}")
print(f"Testcases (unique): {len(test_cases_1_static)}")

In [ ]:
print("Dynamic")
print(f"Testcases (all): {len(test_data_1_dynamic)}")
print(f"Testcases (unique): {len(test_cases_1_dynamic)}")

### Testset 2

Testset 2 is human-generated, for exploring more interesting concepts like emerging technologies, brands and major events (e.g., disease outbreaks and financial crisis). For constructing the test word pairs, we first select emerging terms which have not been popularized before 1994, then query their well known precedents during 1990 to 1994 (e.g., app-2012 can correspond to software-1990).

In [ ]:
test_data_2 = pd.read_csv(path_to_test_data / "testset_2.csv")

In [ ]:
test_data_2.columns = ['truth', 'equivalent']

In [ ]:
test_cases_2 = test_data_2['truth'].unique()

In [ ]:
test_data_2 = test_data_2.sort_values(by='truth', ascending=True)

In [ ]:
test_data_2.head()

In [ ]:
print("Testset 2")
print(f"Testcases (all): {len(test_data_2)}")
print(f"Testcases (unique): {len(test_cases_2)}")

## Models

### TWEC

In [ ]:
model_filenames_cade = glob(str(path_to_twec_model / "*_data.model"))

In [ ]:
model_filenames_cade

In [ ]:
# load models
models_cade = {f"model_{model_file.split('_data')[0][-4:]}":Word2Vec.load(model_file) for model_file in tqdm(model_filenames_cade)}

In [ ]:
models_cade

In [ ]:
models_cade = {model_key: models_cade[model_key] for model_key in sorted(models_cade, key=lambda x: int(x.split('_')[1]))}

In [ ]:
models_cade

In [ ]:
models_cade.keys()

Create dictionary of testsets that contain all test-words along with their embedding in the respective year

In [ ]:
test_case_dict_cade_all = test_util.create_test_case_dict_cade(test_cases_1_all, models_cade)

In [ ]:
test_case_dict_cade_static = test_util.create_test_case_dict_cade(test_cases_1_static, models_cade)

In [ ]:
test_case_dict_cade_dynamic = test_util.create_test_case_dict_cade(test_cases_1_dynamic, models_cade)

Retrieve most similar words for each testword in each year

In [38]:
similarities_cade_all = test_util.get_similarities_of_models(models_cade, test_case_dict_cade_all)

  0%|          | 0/497 [00:00<?, ?it/s]

In [39]:
similarities_cade_static = test_util.get_similarities_of_models(models_cade, test_case_dict_cade_static)

  0%|          | 0/441 [00:00<?, ?it/s]

In [40]:
similarities_cade_dynamic = test_util.get_similarities_of_models(models_cade, test_case_dict_cade_dynamic)

  0%|          | 0/497 [00:00<?, ?it/s]

### Static Word2Vec

In [42]:
model_static = Word2Vec.load(str(path_to_static_model / "w2v_model.model"))

In [43]:
test_case_dict_static_all = test_util.create_test_case_dict_static(model_static, test_cases_1_all)

0 Testcases are not in the vocab of the model


In [44]:
test_case_dict_static_static = test_util.create_test_case_dict_static(model_static, test_cases_1_static)

0 Testcases are not in the vocab of the model


In [45]:
test_case_dict_static_dynamic = test_util.create_test_case_dict_static(model_static, test_cases_1_dynamic)

0 Testcases are not in the vocab of the model


In [46]:
similarities_static_all = test_util.get_similarities_of_models_static(model_static, test_case_dict_static_all)

  0%|          | 0/499 [00:00<?, ?it/s]

In [47]:
similarities_static_static = test_util.get_similarities_of_models_static(model_static, test_case_dict_static_static)

  0%|          | 0/443 [00:00<?, ?it/s]

In [48]:
similarities_static_dynamic = test_util.get_similarities_of_models_static(model_static, test_case_dict_static_dynamic)

  0%|          | 0/499 [00:00<?, ?it/s]

### TPPMI

In [49]:
%autoreload 2

from src.packages.TPPMI.ppmi_model import PPMIModel
from src.packages.TPPMI.tppmi_model import TPPMIModel

UsageError: Line magic function `%autoreload` not found.


In [50]:
ppmi_data_files = sorted(glob(str(path_to_tppmi_model  / "*.npz")))
words_files = sorted(glob(str(path_to_tppmi_model  / "*.pkl")))

Split context-words from timestamped-vocabularies

In [51]:
context_words_file = [path for path in words_files if "context-words" in path]
ppmi_vocab_files = [path for path in words_files if "context-words" not in path]

In [52]:
# Get ppmi-matrices and vocab
ppmi_matrices = {}

for filenames in zip(ppmi_vocab_files, ppmi_data_files):
    ppmi_matrix = sp.load_npz(filenames[1])
    with open(filenames[0], "rb") as f:
        vocab = pickle.load(f)
    key = filenames[0].split("ppmi-")[2][0:4]
    ppmi_matrices[key] = {"ppmi_matrix" : ppmi_matrix, "vocab": vocab}

# Get common context-words
with open(context_words_file[0], "rb") as f:
    context_words = pickle.load(f)

In [53]:
ppmi_matrices.keys()

dict_keys(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'])

Create ppmi_model objects

In [54]:
ppmi_models = {key: PPMIModel.construct_from_data(ppmi_data["ppmi_matrix"], ppmi_data["vocab"], context_words) for key, ppmi_data in ppmi_matrices.items()}

In [55]:
tppmi_model = TPPMIModel(ppmi_models, dates="years")

In [56]:
def create_test_case_dict_tppmi(model: TPPMIModel, test_cases : np.ndarray) -> dict:
    test_case_dict = dict()
    counter = 0
    for test_case in test_cases:
        word, year = test_case.split("-")
        if model.is_in_vocab_of_timestep(word, year):
            df = model.get_tppmi([word])
            df = df[next(iter(df))]
            test_case_dict[test_case] = df.loc[f"{word}_{year}"].to_numpy()
        else:
            counter = counter + 1
    print(f"{counter} Testcases are not in the vocab of the model")

    return test_case_dict

In [57]:
def get_similarites_of_models_tppmi(model: TPPMIModel, test_word_dict: dict) -> dict:
    similarities = dict()
    for word, vector in tqdm(test_word_dict.items()):
        similarities[word] = model.most_similar_words_by_vector(vector)
    return similarities

In [58]:
test_case_dict_tppmi = test_util.create_test_case_dict_tppmi(tppmi_model, test_cases_1_all)

All words are contained in the vocabulary
All words are contained in the vocabulary
All words are contained in the vocabulary
All words are contained in the vocabulary
albright - not in vocab of timestep: 1990
albright - not in vocab of timestep: 1992
albright - not in vocab of timestep: 2013
All words are contained in the vocabulary
albright - not in vocab of timestep: 1990
albright - not in vocab of timestep: 1992
albright - not in vocab of timestep: 2013
All words are contained in the vocabulary
albright - not in vocab of timestep: 1990
albright - not in vocab of timestep: 1992
albright - not in vocab of timestep: 2013
All words are contained in the vocabulary
albright - not in vocab of timestep: 1990
albright - not in vocab of timestep: 1992
albright - not in vocab of timestep: 2013
All words are contained in the vocabulary
athens - not in vocab of timestep: 2006
All words are contained in the vocabulary
All words are contained in the vocabulary
All words are contained in the vocab

In [59]:
# Takes very long, load from .json instead
'''similarities_tppmi = test_util.get_similarites_of_models_tppmi(tppmi_model, test_case_dict_tppmi)

# Writing JSON data
with open(path_to_test_data / 'tppmi-similarities.json', 'w') as file:
    json.dump(similarities_tppmi, file, indent=4)''';

In [60]:
with open(path_to_test_data / 'tppmi-similarities.json', 'r') as file:
    similarities_tppmi_all = json.load(file)

In [61]:
'''test_cade = similarities_cade_all[next(iter(similarities_cade_all))]
test_tppmi = similarities_tppmi[next(iter(similarities_tppmi))]

similarities_tppmi''';

# Experiment

To examine the quality of embedding alignment, we create a task to query equivalences across years.

For example, given obama-2012, we want to query its equivalent word in 2002. As we know obama is the U.S. president in 2012; its equivalent in 2002 is bush, who was the U.S. president at that time. In this way, we create two testsets.

All results are rounded to three decimal places.

In [63]:
cutoffs = [1, 3, 5, 10]
list_of_types = ["static", "dynamic", "all"]

list_of_data_cade = [[similarities_cade_static, test_data_1_static],
                     [similarities_cade_dynamic, test_data_1_dynamic], [similarities_cade_all, test_data_1_all]]
list_of_data_static = [[similarities_static_static, test_data_1_static],
                       [similarities_static_dynamic, test_data_1_dynamic], [similarities_static_all, test_data_1_all]]

config_dict_cade = {key: value for key, value in zip(list_of_types, list_of_data_cade)}
config_dict_static = {key: value for key, value in zip(list_of_types, list_of_data_static)}

scores_cade = {key: dict() for key in list_of_types}
scores_static = {key: dict() for key in list_of_types}
scores_tppmi = {key: dict() for key in list_of_types}

## Mean Reciprocal Rank (@10)

The Mean Reciprocal Rank (MRR) is a statistical measure used to evaluate the performance of a system that returns a ranked list of responses to queries. It is the average of the reciprocal ranks of the first correct answer for each query, where the reciprocal rank is the inverse of the rank at which the first relevant answer is found.
It is evaluated @10

### TWEC

In [64]:
for key, value in tqdm(config_dict_cade.items()):
    scores_cade[key]["mrr@10"] = round(test_util.calculate_rank_metric(value[0], value[1], metric='MRR'), 3)

  0%|          | 0/3 [00:00<?, ?it/s]

### Static Word2Vec

In [65]:
for key, value in tqdm(config_dict_static.items()):
    scores_static[key]["mrr@10"] = round(test_util.calculate_rank_metric_static(value[0], value[1], metric='MRR'), 3)

  0%|          | 0/3 [00:00<?, ?it/s]

### TPPMI

In [66]:
scores_tppmi["all"]["mrr@10"] = round(test_util.calculate_rank_metric_static(similarities_tppmi_all, test_data_1_all, metric='MRR'), 3)

## Mean Precision (@K)

As introduced by Yao et al.(2018) the MP@K is defined as such: consider the K words most similar to the query embedding for the given year. The Precision@K for a particular test i, represented as P@K[i], equals 1 if the target word appears within this set of K words; otherwise, it assumes a value of 0.

### TWEC

In [67]:
for key, value in tqdm(config_dict_cade.items()):
    for k in cutoffs:
        score_key = f"mp@{k}"
        scores_cade[key][score_key] = round(test_util.calculate_rank_metric(value[0], value[1], metric='MP', k=k), 3)

  0%|          | 0/3 [00:00<?, ?it/s]

### Static Word2Vec

In [68]:
for key, value in tqdm(config_dict_static.items()):
    for k in cutoffs:
        score_key = f"mp@{k}"
        scores_static[key][score_key] = round(test_util.calculate_rank_metric_static(value[0], value[1], metric='MP', k=k), 3)

  0%|          | 0/3 [00:00<?, ?it/s]

### TPPMI

In [69]:
for k in cutoffs:
    score_key = f"mp@{k}"
    scores_tppmi["all"][score_key] = round(test_util.calculate_rank_metric_static(similarities_tppmi_all, test_data_1_all, metric='MP', k=k), 3)

## Results

In [70]:
score_table_cade = pd.DataFrame(scores_cade).T
score_table_static = pd.DataFrame(scores_static).T
score_table_tppmi = pd.DataFrame(scores_tppmi).T
print("Scores of TWEC")
display(score_table_cade)
print("Scores of TPPMI")
display(score_table_tppmi)
print("Scores of Static Word2Vec (Baseline)")
display(score_table_static)

Scores of TWEC


,mrr@10,mp@1,mp@3,mp@5,mp@10
static,0.587,0.501,0.656,0.708,0.746
dynamic,0.346,0.268,0.398,0.451,0.512
all,0.394,0.316,0.448,0.501,0.556


Scores of TPPMI


,mrr@10,mp@1,mp@3,mp@5,mp@10
static,NaN,NaN,NaN,NaN,NaN
dynamic,NaN,NaN,NaN,NaN,NaN
all,0.0,0.0,0.0,0.0,0.0


Scores of Static Word2Vec (Baseline)


,mrr@10,mp@1,mp@3,mp@5,mp@10
static,1.000,1.000,1.000,1.000,1.000
dynamic,0.147,0.000,0.240,0.372,0.463
all,0.374,0.266,0.442,0.539,0.606


----------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------